# ****importing all libraries****

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import re
%pylab inline
import matplotlib.image as mpimg
from keras.models import  Sequential
from keras.layers.core import  Lambda , Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D , MaxPooling2D

from keras.preprocessing import image
import tensorflow as tf


# Loading training and test data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'taken for training the model')
print(x_test.shape[0], 'taken for testing')

# this is a multiclass classification problem
*Here multiple classes are present in the y_train part of dataset*

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
print('classes ', y_train.ravel())

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1,2,figsize=(15,5)) 

sns.countplot(y_train.ravel(), ax=ax[0] )
ax[0].set_title("Visualization of training dataset", y=1.01, fontsize=20)
ax[0].set_ylabel("Name of pictures", labelpad=15)
ax[0].set_xlabel("classes of pictures", labelpad=15)

sns.countplot(y_test.ravel(), ax=ax[1] )
plt.title("Visualization of the test dataset", y=1.01, fontsize=20)
plt.ylabel("Name of pictures", labelpad=15)
plt.xlabel("classes of pictures", labelpad=15)


# Convert train datset to (num_images, img_rows, img_cols) format

X_train_dataset = X_train_dataset.reshape(
*                         X_train_dataset.shape[0], 28, 28)
* print(X_train_dataset.shape)
* print(y_train_dataset.shape)

x_test_dataset = x_test_dataset.reshape(x_test_dataset.shape[0], 28, 28)
x_test_dataset.shape

# Visualizing the dataset

In [ ]:

fig=plt.figure(figsize=(10, 10))
columns = 3
rows = 2
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    img = x_train[i]
    plt.imshow(img)
    # if want to show gray image
    # plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
    plt.title(y_train[i])
plt.show()


# Data type conversion
i am converting data to float as it's efficient for computation

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_test.shape

# Preprocessing images

In [ ]:
mean_px = x_train.mean().astype(np.float32)
std_px = x_train.std().astype(np.float32)

def standardize(x): 
    return (x-mean_px)/std_px

# Transform categorical data
**Transforming all the labels into dummy variables as it is a multiclass classification problem.**

In [ ]:
#Normalize

x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

print(y_train[:3])

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(y_train[:3])

In [ ]:
# fixing random seed for reproducibility
seed = 43
np.random.seed(seed)

# Convolutional Neural Network
> making change in images so that the classifier can learn more uniquely

In [ ]:
gen = image.ImageDataGenerator()
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=False,  
        vertical_flip=False)

datagen.fit(x_train)


# **Building CNN model**

In [ ]:
#define the convnet
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Dense, Dropout, Activation, Flatten

from keras.layers import Convolution2D, MaxPooling2D

model6 = Sequential()
model6.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
model6.add(BatchNormalization())
model6.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(Dropout(0.2))
model6.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(Dropout(0.3))
model6.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPool2D((2, 2)))
model6.add(Dropout(0.4))
model6.add(Flatten())
model6.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model6.add(BatchNormalization())
model6.add(Dropout(0.5))
model6.add(Dense(10, activation='softmax'))
# compile model
# opt = SGD(lr=0.001, momentum=0.9)
model6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model = model6
model.summary()

**compiling the model**

*In keras, fit() is much similar to sklearn's fit method, where you pass array of features as x values and target as y values. You pass your whole dataset at once in fit method. Also, use it if you can load whole data into your memory (small dataset).*

In fit_generator(), you don't pass the x and y directly, instead they come from a generator. As it is written in keras documentation, generator is used when you want to avoid duplicate data when using multiprocessing. 

In [ ]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit_generator(datagen.flow(x_train, y_train,
                                batch_size=64),
                                epochs=40,
                              steps_per_epoch=int(x_train.shape[0] / 64),
                                validation_data=(x_test, y_test),
                                workers=4)

In [ ]:
def plotmodelhistory(history): 
    fig, axs = plt.subplots(1,2,figsize=(15,5)) 
    # summarize history for accuracy
    axs[0].plot(history.history['accuracy']) 
    axs[0].plot(history.history['val_accuracy']) 
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy') 
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['train', 'validate'], loc='upper left')
    # summarize history for loss
    axs[1].plot(history.history['loss']) 
    axs[1].plot(history.history['val_loss']) 
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss') 
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['train', 'validate'], loc='upper left')
    plt.show()


plotmodelhistory(history)

# Evaluating the model

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(x_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1)  
Y_pred_classes[:5]

Y_true = np.argmax(y_test, axis=1)


In [ ]:
print(Y_pred_classes[:4])
print(Y_true[:4])

# confusion matrix
**calculating confusion matrix to get the false negative, false positive, true negative and true positive values predicted by the classfier.**

In [ ]:
from sklearn.metrics import confusion_matrix
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
confusion_mtx

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None, cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.
    """
    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)
    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)
    
    ax.set_xlabel('Predicted Label') 
    ax.set_ylabel('True Label')
    
    return im, cbar

def annotate_heatmap(im, data=None, fmt="d", threshold=None):
    """
    A function to annotate a heatmap.
    """
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            text = im.axes.text(j, i, format(data[i, j], fmt), horizontalalignment="center",
                                 color="white" if data[i, j] > thresh else "black")
            texts.append(text)

    return texts

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import matplotlib.pyplot as plt

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 
          'ship', 'truck']
# Errors are difference between predicted labels and true labels
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_test_errors = x_test[errors]

cm = confusion_matrix(Y_true, Y_pred_classes) 
thresh = cm.max() / 2.

fig, ax = plt.subplots(figsize=(12,12))
im, cbar = heatmap(cm, labels, labels, ax=ax,
                   cmap=plt.cm.Blues, cbarlabel="count of predictions")
texts = annotate_heatmap(im, data=cm, threshold=thresh)

fig.tight_layout()
plt.show()

**Classification report**

In [ ]:
print(classification_report(Y_true, Y_pred_classes))


**incorrect prediction**

In [ ]:
R = 2
C = 4
fig, axes = plt.subplots(R, C, figsize=(12,8))
axes = axes.ravel()

misclassified_idx = np.where(Y_pred_classes != Y_true)[0]
for i in np.arange(0, R*C):
    axes[i].imshow(x_test[misclassified_idx[i]])
    axes[i].set_title("True: %s \nPredicted: %s" % (labels[Y_true[misclassified_idx[i]]], 
                                                  labels[Y_pred_classes[misclassified_idx[i]]]))
    axes[i].axis('off')
    plt.subplots_adjust(wspace=1)

**Predicting new value using the trained model.**

In [ ]:
fig = plt.figure(figsize = (3,3))
test_image = np.expand_dims(x_test[26], axis=0)
test_result = model.predict_classes(test_image)
plt.imshow(x_test[26])
dict_key = test_result[0]
plt.title("Predicted: {} \nTrue Label: {}".format(labels[dict_key], labels[Y_true[26]]))

**saving the prediction to a csv file**

In [ ]:
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_trained_model.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# checking the model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
Y_pred_classes[:2]

In [ ]:
!pip install py7zr
from py7zr import unpack_7zarchive
import shutil
import os

shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

In [ ]:
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z', '/kaggle/working')

In [ ]:
test_dir = os.listdir("./test")
test_dir_len = len(test_dir)

print(".\\test:\t",test_dir_len)
print("files:\t\t",test_dir[:3])



In [ ]:
test_data_generator = ImageDataGenerator(rescale=1./255.)
test_generator = test_data_generator.flow_from_directory(directory='/kaggle/working',
            batch_size=64,
            shuffle=False,color_mode='rgb',
            target_size=(32,32),
            class_mode=None)



In [ ]:
test_prediction = model.predict_generator(test_generator)

predicted_class = np.argmax(test_prediction, axis=1)

In [ ]:
print(predicted_class[:3])
predicted_class.shape


In [ ]:
submission1 = [labels[i] for i in predicted_class]

submission = pd.DataFrame({"id": list(range(1, len(predicted_class)+1)),
                          "label": submission1})

submission.to_csv("submission.csv", index=False)
submission[:2]

In [ ]:
var = pd.read_csv("./submission.csv")
var.shape

In [ ]:
var.head(101)

In [ ]:
import matplotlib.image as mpimg
fig = plt.figure(figsize = (3,3))
img = mpimg.imread("/kaggle/working/test/"+ str(test_dir[1]))
plt.imshow(img)

In [ ]:
index = 0    
fig = plt.figure(figsize = (16,10))
for item in submission.values[50:70]:
    index += 1
    plt.subplot(5, 5, index)
    test_path = '/kaggle/working/test/'+str(item[0])+'.png'
    print(test_path)
    test_image = image.load_img(test_path, target_size=(32,32))
    plt.imshow(test_image)
    plt.colorbar()
    plt.grid(False)
    plt.axis("off")
    
    test_result = model.predict_classes(test_image)
    dict_key = test_result[0]
    plt.title(labels[dict_key])
plt.show()


# test_result = model.predict_classes(test_image)
# plt.imshow(x_test[26])
# dict_key = test_result[0]
# plt.title("Predicted: {} \nTrue Label: {}".format(labels[dict_key], labels[Y_true[26]]))